# MTTN to MMN
## Input: Multilayer temporal text network created in file 'ttn_to_mttn'
## Output: Various multilayer message networks, where layers represents social dimensions and sentiment

In [1]:
import uunet.multinet as ml
import pandas as pd

In [2]:
def initialise_net(messages, ttn, directed):
    '''
    initialises net
    '''
    # get layers
    layers = ml.layers(ttn)
    layers.remove('persons')

    # create empty net
    messages_10d = ml.empty()

    # add layers
    ml.add_layers(messages_10d, layers=layers, directed=[directed]*len(layers))
    
    # add messages to each laye
    for layer in layers:
        message_vertices = {"actor": messages, "layer": [layer]*len(messages)}
        ml.add_vertices(messages_10d, message_vertices)

    return messages_10d
    
def project(ttn, edges_df, messages, layer, messages_10d, time_constraint, time_constraint_activated):
    '''
    project to layer
    '''

    # fill message network
    for message in messages:
        
        # get message-receivers
        receivers = list(edges_df.loc[edges_df['from_actor'] == message]['to_actor'])
        if len(receivers) == 0:
            continue
            
        # get messages sent by receivers
        sent_by_receivers = list(edges_df.loc[edges_df['from_actor'].isin(receivers)]['to_actor'])
        
        # skip loop if there are no receivers
        if len(sent_by_receivers) == 0:
            continue

        # get time
        end_message = ml.get_values(ttn, "end", vertices = {'actor': [message], 'layer':['messages']})['end'][0]
        start_times = ml.get_values(ttn, "start", vertices = {'actor': sent_by_receivers, 'layer':['messages']*len(sent_by_receivers)})['start']
    
        # remove message that is sent by receivers if they dont fit the time constraint 
        remove = []
        if time_constraint_activated:
            for i, start in enumerate(start_times):
                dif = start-end_message
                if dif > time_constraint or dif < 0:
                    remove.append(sent_by_receivers[i])
            sent_by_receivers = [m for m in sent_by_receivers if m not in remove]

        # add edges between messages
        edges = {"from_actor": [message]*len(sent_by_receivers), "from_layer": [layer]*len(sent_by_receivers), 
                 "to_actor": sent_by_receivers, "to_layer": [layer]*len(sent_by_receivers)}

        # add edges
        ml.add_edges(messages_10d, edges) 


In [4]:
#main

# start constraint
start_threshold = 10

# net types
net_types = [{'directed':True, 'time':True},{'directed':True, 'time':False},
             {'directed':False, 'time':True},{'directed':False, 'time':False}] # specify which types of networks that will be created

# load ttn
ttn = ml.read('data/ttn_nets/mttn.txt')

# get layers to project from
layers = ml.layers(ttn)
layers.remove('persons')

# get all messages
messages = ml.vertices(ttn, layers=['messages'])['actor']

# create one 10d-messages net for each type
for net_type in net_types:

    # unpack type
    directed = net_type['directed']
    time_constraint_activated = net_type['time']
    
    # initialise messages net 
    messages_10d = initialise_net(messages, ttn, directed)

    # name
    name = 'mmn'
    if directed:
        name = name + '_directed'
    else:
        name = name + '_undirected'
    if time_constraint_activated:
        name = name + '_time'

    # project on to each layer    
    for layer in layers:

        # get messages
        layer_messages = ml.vertices(ttn, layers=[layer])['actor']

        # get edges from original 10d-ttn
        edges = ml.edges(ttn, layers1=['persons',layer])
        edges_df = pd.DataFrame.from_dict(edges)

        # project 
        project(ttn, edges_df,layer_messages,layer,messages_10d, start_threshold, time_constraint_activated)
   
    # save 
    save_path = f'data/multilayer_message_networks/{name}'
    ml.write(n = messages_10d,file = save_path+'.graphml', format = "graphml")
    ml.write(n = messages_10d,file = save_path+'.txt', format = "multilayer")

    print(name)
    display(pd.DataFrame.from_dict(ml.summary(messages_10d)))
      

mmn_directed_time


,layer,n,m,dir,nc,slc,dens,cc,apl,dia
0,romance,1064,4,True,1064,1,3.536593e-06,0.000000,0,0
1,conflict,1064,25,True,1064,1,2.210371e-05,0.000000,0,0
2,negative,1064,123,True,1064,1,1.087502e-04,0.041667,0,0
3,power,1064,6,True,1064,1,5.304890e-06,0.000000,0,0
4,identity,1064,0,True,1064,1,0.000000e+00,0.000000,0,0
5,fun,1064,5,True,1064,1,4.420741e-06,0.000000,0,0
6,support,1064,15,True,1064,1,1.326222e-05,0.500000,0,0
7,messages,1064,1527,True,1064,1,1.350094e-03,0.188557,0,0
8,knowledge,1064,16,True,1064,1,1.414637e-05,0.000000,0,0
9,status,1064,1,True,1064,1,8.841483e-07,0.000000,0,0


mmn_directed


,layer,n,m,dir,nc,slc,dens,cc,apl,dia
0,negative,1064,12569,True,809,256,0.011113,0.074909,2.165594,5
1,power,1064,471,True,1028,37,0.000416,0.060390,2.045796,5
2,romance,1064,480,True,1020,45,0.000424,0.044586,2.262121,5
3,conflict,1064,3061,True,947,118,0.002706,0.071561,2.126105,6
4,knowledge,1064,1714,True,986,75,0.001515,0.033343,2.156937,5
5,support,1064,1522,True,988,77,0.001346,0.040334,2.205571,5
6,messages,1064,194262,True,55,994,0.171756,0.080501,2.129283,5
7,fun,1064,72,True,1056,5,0.000064,0.143357,1.700000,3
8,identity,1064,0,True,1064,1,0.000000,0.000000,0.000000,0
9,neutral,1064,55528,True,526,526,0.049095,0.070641,2.149766,5


mmn_undirected_time


,layer,n,m,dir,nc,slc,dens,cc,apl,dia
0,support,1064,15,False,1050,3,0.000027,0.500000,1.333333,2
1,romance,1064,4,False,1060,5,0.000007,0.000000,2.000000,4
2,conflict,1064,25,False,1040,4,0.000044,0.000000,1.333333,2
3,negative,1064,123,False,946,7,0.000218,0.034091,2.285714,4
4,power,1064,6,False,1058,3,0.000011,0.000000,1.333333,2
5,identity,1064,0,False,1064,1,0.000000,0.000000,0.000000,0
6,messages,1064,1527,False,175,49,0.002700,0.255966,7.515306,22
7,knowledge,1064,16,False,1048,4,0.000028,0.000000,1.666667,3
8,fun,1064,5,False,1059,3,0.000009,0.000000,1.333333,2
9,status,1064,1,False,1063,2,0.000002,0.000000,1.000000,1


mmn_undirected


,layer,n,m,dir,nc,slc,dens,cc,apl,dia
0,romance,1064,437,False,1008,57,0.000773,0.118552,1.861529,4
1,messages,1064,176185,False,7,1055,0.311547,0.156003,1.731745,4
2,negative,1064,10998,False,802,263,0.019448,0.133926,1.749630,4
3,power,1064,412,False,1016,49,0.000729,0.133033,1.728741,3
4,conflict,1064,2633,False,940,125,0.004656,0.137081,1.737419,4
5,support,1064,1401,False,974,91,0.002477,0.084335,1.707692,3
6,knowledge,1064,1542,False,967,98,0.002727,0.071228,1.804334,4
7,identity,1064,0,False,1064,1,0.000000,0.000000,0.000000,0
8,similarity,1064,9,False,1055,8,0.000016,0.000000,2.214286,4
9,fun,1064,65,False,1042,21,0.000115,0.235294,1.909524,3
